In [1]:
import os
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma

# fetch the api key from the environment variables
api_key = os.environ["OPENAI_API_KEY"]

# Set OpenAI API Key (ensure you have an API key set in env variables)
os.environ["OPENAI_API_KEY"] = api_key

# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

# Split text into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Initialize OpenAI embeddings and ChromaDB
embedding_model = OpenAIEmbeddings()
vector_store = Chroma.from_documents(all_splits, embedding_model)

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")

# Initialize OpenAI Chat Model
llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}

def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

question = "What are the key components of an AI agent according to the blog post?"
result = graph.invoke({"question": question})
print(result["answer"])

USER_AGENT environment variable not set, consider setting it to identify your requests.


The key components of an AI agent, according to the blog post, include Planning, Task Decomposition, and API Tool Use Capabilities. Planning involves understanding and planning ahead for complex tasks. Task Decomposition uses the Chain of Thought technique to break down complex tasks into simpler steps, and API Tool Use Capabilities are evaluated at three levels: the ability to call, retrieve, and plan API usage beyond simple retrieval and calling.
